In [2]:
!pip install -q darts


In [19]:
import pandas as pd
import numpy as np

from darts import TimeSeries
from darts.models import LinearRegressionModel
from darts.metrics import mse
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries


In [20]:
df = pd.DataFrame(pd.read_csv('../raw_data/final_dataset.csv'))
df['index'] = pd.to_datetime(df['index'])
df = df.set_index('index')


In [18]:
df.head


<bound method NDFrame.head of                      traditional_baguette  croissant  coupe  pain_au_chocolat  \
index                                                                           
2021-01-01 00:00:00                   0.0        0.0    0.0               0.0   
2021-01-01 00:20:00                   0.0        0.0    0.0               0.0   
2021-01-01 00:40:00                   0.0        0.0    0.0               0.0   
2021-01-01 01:00:00                   0.0        0.0    0.0               0.0   
2021-01-01 01:20:00                   0.0        0.0    0.0               0.0   
...                                   ...        ...    ...               ...   
2022-09-30 21:40:00                   0.0        0.0    0.0               0.0   
2022-09-30 22:00:00                   0.0        0.0    0.0               0.0   
2022-09-30 22:20:00                   0.0        0.0    0.0               0.0   
2022-09-30 22:40:00                   0.0        0.0    0.0               0.0  

In [21]:
target = df[['traditional_baguette', 'croissant', 'coupe', 'pain_au_chocolat', 'baguette', 'banette', 'cereal_baguette']]
target_tradi = df[['traditional_baguette']]
past_cov = df[['temperature_2m (°C)', 'relative_humidity_2m (%)', 'apparent_temperature (°C)', 'precipitation (mm)', 'rain (mm)', 'wind_speed_10m (km/h)', 'wind_speed_100m (km/h)', 'day_of_week_sin', 'day_of_week_cos', 'month_sin', 'month_cos', 'isHoliday']]
#future_covariate =


In [22]:
df_series = TimeSeries.from_dataframe(df)
target_series = TimeSeries.from_dataframe(target)
target_tradi_series = TimeSeries.from_dataframe(target_tradi)
past_cov_series = TimeSeries.from_dataframe(past_cov)


In [23]:
lags = 7 * 72
lags_past_covariates = 7 * 72
output_chunk_lenght = 7 * 72


In [24]:
# We first set aside the first 80% as training series:
target_series_train, _ = target_series.split_before(0.8)

def eval_model(model, past_covariates = None,
               future_covariates = None):



    backtest = model.historical_forecasts(series = target_series,
                                          past_covariates = past_covariates,
                                          #future_covariates = future_covariates,
                                          start = 0.8,
                                          retrain = False,
                                          verbose = True,
                                          forecast_horizon = output_chunk_lenght)

    df_series[-len(backtest)-100:].plot()
    backtest.plot(label = 'backtest (n = 10)')
    print('Backtest MSE = {}'.format(mse(target_series, backtest)))


In [25]:
lr = LinearRegressionModel(lags = lags,
                             lags_past_covariates = lags_past_covariates,
                             #lags_future_covariates=[-4, -3, -2, -1, 0]
)

lr.fit(target_series,
               past_covariates = past_cov_series,
               #future_covariates=rainfalls
)

eval_model(lr)


: 